<a href="https://colab.research.google.com/github/Anushil007/Review-Classifier/blob/main/Comment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from pyexpat import model
import nltk
from nltk.corpus import stopwords
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import string



In [2]:
path = '/content/drive/MyDrive/Comment Classifier/Hotel_Reviews.csv'
datafr = pd.read_csv(path)


In [3]:
df = datafr[['Negative_Review','Positive_Review']]

In [4]:
'''
Drop Comments like No Positive or No Negative
'''

df.drop(df[df['Positive_Review'] == 'No Positive'].index,inplace=True)
df.drop(df[df['Negative_Review'] == 'No Negative'].index,inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
'''
Assign Label 0 to Negative Reviews
'''

df1 = pd.DataFrame(columns = ['Text','Label'])
df1['Text'] = df['Negative_Review']
df1['Label'] = 0
#print(df1.head())


In [6]:
'''
Assign Label 1 to Positive Reviews
'''

df2= pd.DataFrame(columns = ['Text','Label'])
df2['Text'] = df['Positive_Review']
df2['Label'] = 1
#print(df2.head())



In [7]:
'''
Combine 2 DataFrames
'''
frames = [df1,df2]
final_df = pd.concat(frames)
#print(final_df.tail())

final_df.to_csv('Total_Reviews.csv',index=False)


In [9]:
nltk.download('stopwords')
vectorizer = CountVectorizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def pre_process_df():
    '''
    Import CSV file and store it in a DataFrame to use.
    '''
    f_df = pd.read_csv('/content/drive/MyDrive/Comment Classifier/Total_Reviews.csv')
    df = pd.DataFrame(columns=['Text','Label'])

    df['Text'] = f_df['Text']
    df['Label'] = f_df['Label']
    return df


In [11]:
def input_process(text):
    '''
    Remove Stop Words.
    '''
    translator = str.maketrans('','',string.punctuation)
    nopunc = text.translate(translator)
    words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return ' '.join(words)


In [12]:
def remove_stop_words(input):
    final_input = []
    for line in input:
        line = input_process(line)
        final_input.append(line)
    return final_input


In [13]:
def train_model(df):
    '''
    Define a model and train its transfromed form from the vectorizer.
    '''
    input = df['Text']
    output = df['Label']
    input = remove_stop_words(input)
    df['Text'] = input
    input = vectorizer.fit_transform(input)
    nb = MultinomialNB()
    nb.fit(input,output)
    return nb


In [14]:
df = pre_process_df()
model = train_model(df)
pickle.dump(model,open('classifier.model','wb'))
pickle.dump(vectorizer,open('vectorizer.pickle','wb'))

In [23]:
def load_model_and_vectorizer():
    '''
    Load model and vectorizer.
    '''
    model = pickle.load(open('/content/drive/MyDrive/Comment Classifier/classifier.model','rb'))
    vectorizer = pickle.load(open('/content/drive/MyDrive/Comment Classifier/vectorizer.pickle','rb'))
    return model, vectorizer


In [51]:
model,vectorizer = load_model_and_vectorizer()
content = input('Enter Review you want to Classify:')
content = input_process(content)

Enter Review you want to Classify:Good product but installation cost additional Rs 3000 for labour, wire,stand ,plug. Dealers need to improve in providing quick solutions to issues during installation due to height in apartment. Had to postpone installation by one day since installing team didn't have all tools on day one. But installing was done good even after delay. Have check the performance of AC now


In [52]:
content_vec = vectorizer.transform([content])
pred = model.predict(content_vec)
if pred[0] == 1:
        print('Good Review!')
else:
        print('Bad Review!')

Bad Review!


TextBlob is used to determine the attitude or the emotion of the writer<br>
subjectivity [0:1] is the amount of personal opinion and factual info contained in the text<br>
higher value of subjectivity means more personal opinion rather than factual info.<br>
polarity [-1:1] defines how positive a statement is. 1 is positive and -1 is negative<br>


In [53]:
from textblob import TextBlob
Subjectivity = TextBlob(content).sentiment.subjectivity
Polarity = TextBlob(content).sentiment.polarity

In [54]:
Subjectivity,Polarity

(0.51875, 0.4020833333333333)

In [55]:
if float(pred[0] == 1) and Polarity >= 0 or float(pred[0] == 0) and Polarity < 0:
  print('Correct Prediction')
else:
  print('Incorrect Prediction')

Incorrect Prediction
